# Homework #1: Predicting Fuel Consumption

### Machine Learning and Computational Statistics (DSC6232)

#### Instructors: Weiwei Pan, Melanie Fernandez, Pavlos Protopapas

#### Due: June 25th, 2:00 pm Kigali Time

In [1]:
# as usual, let us load all the necessary libraries
import numpy as np  # numerical computation with arrays
import pandas as pd # library to manipulate datasets using dataframes
import scipy as sp  # statistical library
#from google.colab import files
from sklearn.model_selection import train_test_split

# below sklearn libraries for different models
from sklearn.tree import DecisionTreeClassifier as DecisionTree
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier as RandomForest
from sklearn.linear_model import Ridge, Lasso
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

#import libraries for implementing neural networks
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD, Adam
from keras.regularizers import l2

# plot 
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

ModuleNotFoundError: No module named 'keras'

# Loading Competition Data
Run all the steps below to obtain the data

In [2]:
# Read training data
train_data = pd.read_csv('https://raw.githubusercontent.com/onefishy/Rwanda-course-2020/master/Competition_data/train.csv') # read in the data as a DataFrame
train_data.head(3) # show the first 3 rows of the dataset

,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name,fuel (L/100km)
0,4,106.0,63.00,2123.0,14.7,82,1,"""amc hornet""",6.189737
1,8,400.0,150.0,3760.0,8.5,70,1,"""dodge challenger se""",15.680667
2,4,104.0,70.00,2150.0,13.9,79,1,"""honda civic cvcc""",6.817681


In [3]:
d = train_data.describe().T
d[['mean', 'max', 'min']]

,mean,max,min
cylinders,5.412752,8.000000,3.000000
displacement,189.921141,455.000000,67.000000
weight,2769.526846,5141.000000,-4294.000000
acceleration,15.643624,24.700000,8.500000
model year,76.013423,82.000000,70.000000
origin,1.577181,3.000000,1.000000
fuel (L/100km),11.151093,26.134444,5.273767


In [4]:
train_data.isna().sum()
#train_data.dropna()

cylinders         0
displacement      0
horsepower        0
weight            0
acceleration      0
model year        0
origin            0
car name          0
fuel (L/100km)    0
dtype: int64

In [8]:
train_data.dtypes

cylinders           int64
displacement      float64
horsepower         object
weight            float64
acceleration      float64
model year          int64
origin              int64
car name           object
fuel (L/100km)    float64
dtype: object

In [9]:
print(type(train_data))

<class 'pandas.core.frame.DataFrame'>


In [10]:
df = pd.DataFrame(train_data)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 298 entries, 0 to 297
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   cylinders       298 non-null    int64  
 1   displacement    298 non-null    float64
 2   horsepower      298 non-null    object 
 3   weight          298 non-null    float64
 4   acceleration    298 non-null    float64
 5   model year      298 non-null    int64  
 6   origin          298 non-null    int64  
 7   car name        298 non-null    object 
 8   fuel (L/100km)  298 non-null    float64
dtypes: float64(4), int64(3), object(2)
memory usage: 21.1+ KB


In [12]:
df.columns

Index(['cylinders', 'displacement', 'horsepower', 'weight', 'acceleration',
       'model year', 'origin', 'car name', 'fuel (L/100km)'],
      dtype='object')

In [ ]:
list(df.columns)

['cylinders',
 'displacement',
 'horsepower',
 'weight',
 'acceleration',
 'model year',
 'origin',
 'car name',
 'fuel (L/100km)']

In [14]:
# Read test data 
test_data = pd.read_csv('https://raw.githubusercontent.com/onefishy/Rwanda-course-2020/master/Competition_data/test.csv') # read in the data as a DataFrame
test_data.head(3) # show the first 3 rows of the dataset

,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
0,4,122.0,80.0,2450.0,15.5,74,1,"""ford galaxie 500"""
1,6,257.0,95.0,3191.0,17.8,76,1,"""ford pinto"""
2,4,87.0,65.0,2108.0,18.9,80,3,"""plymouth duster"""


In [15]:
list(test_data.columns)

['cylinders',
 'displacement',
 'horsepower',
 'weight',
 'acceleration',
 'model year',
 'origin',
 'car name']

In [18]:
test_data.isna().sum()

cylinders       0
displacement    0
horsepower      0
weight          0
acceleration    0
model year      0
origin          0
car name        0
dtype: int64

In [19]:
len(test_data)

100

In [20]:
len(df)

298

In [21]:
print(len(df['model year']))
print(len(df['model year'].unique()))
print(len(df['model year']) - len(df['model year'].unique()))

298
13
285


In [22]:
# Step 1: one hot encode the categorical covariate 'nationality'
one_hot_model_year = pd.get_dummies(df['model year'], prefix='model year')

# Step 2: use pd.concat to join the new columns with your original dataframe
df = pd.concat([df, one_hot_model_year], axis=1)

# Stpe 3: now drop the original 'country' column (you don't need it anymore)
df.drop(['model year'], axis=1, inplace=True)

# Step 4: print the resultant dataframe
df.head(2)

,cylinders,displacement,horsepower,weight,acceleration,origin,car name,fuel (L/100km),model year_70,model year_71,...,model year_73,model year_74,model year_75,model year_76,model year_77,model year_78,model year_79,model year_80,model year_81,model year_82
0,4,106.0,63.00,2123.0,14.7,1,"""amc hornet""",6.189737,0,0,...,0,0,0,0,0,0,0,0,0,1
1,8,400.0,150.0,3760.0,8.5,1,"""dodge challenger se""",15.680667,1,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
# Step 1: one hot encode the categorical covariate 'nationality'
one_hot_model_year = pd.get_dummies(test_data['model year'], prefix='model year')

# Step 2: use pd.concat to join the new columns with your original dataframe
test_data = pd.concat([test_data, one_hot_model_year], axis=1)

# Stpe 3: now drop the original 'country' column (you don't need it anymore)
test_data.drop(['model year'], axis=1, inplace=True)

In [24]:
print(len(df['origin']))
print(len(df['origin'].unique()))
print(len(df['origin']) - len(df['origin'].unique()))

298
3
295


In [25]:
# # Step 1: one hot encode the categorical covariate 'nationality'
# one_hot_origin = pd.get_dummies(df['origin'], prefix='origin')

# # Step 2: use pd.concat to join the new columns with your original dataframe
# df = pd.concat([df, one_hot_origin], axis=1)

# # Stpe 3: now drop the original 'country' column (you don't need it anymore)
# df.drop(['origin'], axis=1, inplace=True)

# # Step 4: print the resultant dataframe
# df.head(2)

In [26]:
# # Step 1: one hot encode the categorical covariate 'nationality'
# one_hot_origin = pd.get_dummies(test_data['origin'], prefix='origin')

# # Step 2: use pd.concat to join the new columns with your original dataframe
# test_data = pd.concat([test_data, one_hot_origin], axis=1)

# # Stpe 3: now drop the original 'country' column (you don't need it anymore)
# test_data.drop(['origin'], axis=1, inplace=True)

# # Step 4: print the resultant dataframe
# test_data.head(2)

In [27]:

one_hot_cylinders = pd.get_dummies(test_data['cylinders'], prefix='cylinders')


test_data = pd.concat([test_data, one_hot_cylinders], axis=1)

test_data.drop(['cylinders'], axis=1, inplace=True)


test_data.head(2)

,displacement,horsepower,weight,acceleration,origin,car name,model year_70,model year_71,model year_72,model year_73,...,model year_77,model year_78,model year_79,model year_80,model year_81,model year_82,cylinders_3,cylinders_4,cylinders_6,cylinders_8
0,122.0,80.0,2450.0,15.5,1,"""ford galaxie 500""",0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,257.0,95.0,3191.0,17.8,1,"""ford pinto""",0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [28]:
one_hot_cylinders = pd.get_dummies(df['cylinders'], prefix='cylinders')


df = pd.concat([df, one_hot_cylinders], axis=1)

df.drop(['cylinders'], axis=1, inplace=True)


df.head(2)

,displacement,horsepower,weight,acceleration,origin,car name,fuel (L/100km),model year_70,model year_71,model year_72,...,model year_78,model year_79,model year_80,model year_81,model year_82,cylinders_3,cylinders_4,cylinders_5,cylinders_6,cylinders_8
0,106.0,63.00,2123.0,14.7,1,"""amc hornet""",6.189737,0,0,0,...,0,0,0,0,1,0,1,0,0,0
1,400.0,150.0,3760.0,8.5,1,"""dodge challenger se""",15.680667,1,0,0,...,0,0,0,0,0,0,0,0,0,1


In [29]:
print(len(df['car name']))
print(len(df['car name'].unique()))
print(len(df['car name']) - len(df['car name'].unique()))

298
237
61


In [30]:
# Stpe 3: now drop the car name variable because it has no importance
df.drop(['car name'], axis=1, inplace=True)

In [31]:
test_data.drop(['car name'], axis=1, inplace=True)

In [32]:
df.drop(['cylinders_5'], axis=1, inplace=True)

In [33]:
# df.drop(['cylinders_8'], axis=1, inplace=True)

In [34]:
# test_data.drop(['cylinders_8'], axis=1, inplace=True)

In [35]:
df.drop(['cylinders_3'], axis=1, inplace=True)

In [36]:
test_data.drop(['cylinders_3'], axis=1, inplace=True)

In [37]:
df.drop(['cylinders_6'], axis=1, inplace=True)

In [38]:
test_data.drop(['cylinders_6'], axis=1, inplace=True)

In [39]:
df.drop(['origin'], axis=1, inplace=True)

In [40]:
test_data.drop(['origin'], axis=1, inplace=True)

In [41]:
print(len(df['horsepower']))
print(len(df['horsepower'].unique()))
print(len(df['horsepower']) - len(df['horsepower'].unique()))

298
87
211


In [42]:
# Step 1: Remove
df['horsepower'].unique()

array(['   63.00  ', '   150.0  ', '   70.00  ', '   68.00  ',
       '   120.0  ', '   49.00  ', '   97.00  ', '   74.00  ',
       '   88.00  ', '   170.0  ', '   90.00  ', '   130.0  ',
       '   140.0  ', '   165.0  ', '   115.0  ', '   78.00  ',
       '   81.00  ', '   180.0  ', '   95.00  ', '   75.00  ',
       '   ?      ', '   85.00  ', '   125.0  ', '   190.0  ',
       '   52.00  ', '   198.0  ', '   87.00  ', '   100.0  ',
       '   83.00  ', '   64.00  ', '   110.0  ', '   108.0  ',
       '   72.00  ', '   67.00  ', '   53.00  ', '   105.0  ',
       '   80.00  ', '   98.00  ', '   91.00  ', '   60.00  ',
       '   142.0  ', '   62.00  ', '   77.00  ', '   132.0  ',
       '   175.0  ', '   148.0  ', '   71.00  ', '   145.0  ',
       '   46.00  ', '   93.00  ', '   122.0  ', '   139.0  ',
       '   79.00  ', '   84.00  ', '   215.0  ', '   76.00  ',
       '   65.00  ', '   133.0  ', '   92.00  ', '   86.00  ',
       '   220.0  ', '   137.0  ', '   61.00  ', '   69

In [43]:
test_data['horsepower'].unique()

array([ 80.,  95.,  65., 167., 150., 110.,  52.,  85.,  46.,  83.,  88.,
        67., 155.,  72., 100.,  92.,  90., 170.,  63., 135.,  53.,  84.,
       120., 225.,  89.,  78., 165.,  70.,  75., 125., 129., 140.,  62.,
        58.,  97.,  68., 105.,  71., 153.,  96., 145., 112., 138., 160.,
        82.,  48., 116., 175., 190.])

In [44]:
#df['horsepower'] = df['horsepower'].str.strip()

In [45]:
df['horsepower'] = pd.to_numeric(df['horsepower'],errors='coerce')


In [46]:
df.horsepower.head(59)

0      63.0
1     150.0
2      70.0
3      68.0
4     120.0
5      49.0
6      97.0
7      74.0
8      88.0
9     170.0
10     90.0
11    130.0
12     70.0
13    140.0
14    165.0
15    115.0
16     78.0
17     68.0
18     81.0
19    180.0
20    150.0
21     95.0
22     75.0
23      NaN
24     85.0
25    125.0
26    140.0
27    190.0
28     52.0
29    190.0
30    198.0
31     70.0
32     88.0
33     87.0
34    180.0
35    100.0
36     83.0
37     90.0
38    140.0
39     75.0
40     64.0
41    110.0
42    170.0
43    110.0
44    108.0
45     72.0
46      NaN
47     67.0
48     88.0
49     53.0
50    105.0
51     80.0
52    100.0
53     68.0
54    140.0
55     88.0
56     90.0
57     88.0
58     90.0
Name: horsepower, dtype: float64

In [47]:
df.horsepower.head(59)

0      63.0
1     150.0
2      70.0
3      68.0
4     120.0
5      49.0
6      97.0
7      74.0
8      88.0
9     170.0
10     90.0
11    130.0
12     70.0
13    140.0
14    165.0
15    115.0
16     78.0
17     68.0
18     81.0
19    180.0
20    150.0
21     95.0
22     75.0
23      NaN
24     85.0
25    125.0
26    140.0
27    190.0
28     52.0
29    190.0
30    198.0
31     70.0
32     88.0
33     87.0
34    180.0
35    100.0
36     83.0
37     90.0
38    140.0
39     75.0
40     64.0
41    110.0
42    170.0
43    110.0
44    108.0
45     72.0
46      NaN
47     67.0
48     88.0
49     53.0
50    105.0
51     80.0
52    100.0
53     68.0
54    140.0
55     88.0
56     90.0
57     88.0
58     90.0
Name: horsepower, dtype: float64

In [48]:
# df['horsepower'].fillna((df['horsepower'].mean()), inplace=True)
df['horsepower'] = df['horsepower'].fillna(85)

In [49]:
df.horsepower.head(50)

0      63.0
1     150.0
2      70.0
3      68.0
4     120.0
5      49.0
6      97.0
7      74.0
8      88.0
9     170.0
10     90.0
11    130.0
12     70.0
13    140.0
14    165.0
15    115.0
16     78.0
17     68.0
18     81.0
19    180.0
20    150.0
21     95.0
22     75.0
23     85.0
24     85.0
25    125.0
26    140.0
27    190.0
28     52.0
29    190.0
30    198.0
31     70.0
32     88.0
33     87.0
34    180.0
35    100.0
36     83.0
37     90.0
38    140.0
39     75.0
40     64.0
41    110.0
42    170.0
43    110.0
44    108.0
45     72.0
46     85.0
47     67.0
48     88.0
49     53.0
Name: horsepower, dtype: float64

In [50]:
# # Step 1: define an knn imputer using KNNImputer
# # Hint: look up the documentation for KNNImputer
# # COMPLETE
# imputer = KNNImputer(n_neighbors=2)

# # Step 2: impute the missing data
# df = pd.DataFrame(imputer.fit_transform(df), columns=df.columns, index=df.index)

In [51]:
list(df.columns)

['displacement',
 'horsepower',
 'weight',
 'acceleration',
 'fuel (L/100km)',
 'model year_70',
 'model year_71',
 'model year_72',
 'model year_73',
 'model year_74',
 'model year_75',
 'model year_76',
 'model year_77',
 'model year_78',
 'model year_79',
 'model year_80',
 'model year_81',
 'model year_82',
 'cylinders_4',
 'cylinders_8']

In [52]:
df['weight'] = df['weight'].abs()
df['weight'] 

0      2123.0
1      3760.0
2      2150.0
3      1971.0
4      3819.0
        ...  
293    4383.0
294    2335.0
295    2288.0
296    3610.0
297    2045.0
Name: weight, Length: 298, dtype: float64

In [53]:
test_data.head(5)

,displacement,horsepower,weight,acceleration,model year_70,model year_71,model year_72,model year_73,model year_74,model year_75,model year_76,model year_77,model year_78,model year_79,model year_80,model year_81,model year_82,cylinders_4,cylinders_8
0,122.0,80.0,2450.0,15.5,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
1,257.0,95.0,3191.0,17.8,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,87.0,65.0,2108.0,18.9,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
3,399.0,167.0,4906.0,12.5,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
4,399.0,150.0,4462.0,13.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1


In [54]:
# # Step 1: define a MinMax scalar that will transform the data values into values in (0, 1)
# # Hint: Look up the documentation for MinMaxScaler
# # COMPLETE
# scaler = MinMaxScaler()
# # scalar = MinMaxScaler()

# # Step 2: fit the MinMaxScaler using our data 
# # Hint: use .fit()
# # COMPLETE
# scaler.fit(df)
# X_values = df.values
# # Step 3: scale the values in our dataset, X_values 
# # Hint: use .transform()
# # COMPLETE
# X_values_scaled = scaler.transform(X_values)
# df = pd.DataFrame(X_values_scaled, columns=df.columns)

In [55]:
# # Step 1: define a MinMax scalar that will transform the data values into values in (0, 1)
# # Hint: Look up the documentation for MinMaxScaler
# # COMPLETE
# scaler = MinMaxScaler()
# # scalar = MinMaxScaler()

# # Step 2: fit the MinMaxScaler using our data 
# # Hint: use .fit()
# # COMPLETE
# scaler.fit(test_data)
# X_values1 = test_data.values
# # Step 3: scale the values in our dataset, X_values 
# # Hint: use .transform()
# # COMPLETE
# X_values_scaled1 = scaler.transform(X_values1)
# test_data = pd.DataFrame(X_values_scaled1, columns=test_data.columns)

In [56]:
#Split the train data into target and features
# create features
x_train = df.loc[:, df.columns != 'fuel (L/100km)']
x_train

,displacement,horsepower,weight,acceleration,model year_70,model year_71,model year_72,model year_73,model year_74,model year_75,model year_76,model year_77,model year_78,model year_79,model year_80,model year_81,model year_82,cylinders_4,cylinders_8
0,106.0,63.0,2123.0,14.7,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
1,400.0,150.0,3760.0,8.5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,104.0,70.0,2150.0,13.9,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
3,92.0,68.0,1971.0,17.6,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
4,167.0,120.0,3819.0,16.7,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293,399.0,175.0,4383.0,12.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
294,89.0,48.0,2335.0,24.7,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
295,106.0,72.0,2288.0,17.0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
296,341.0,160.0,3610.0,9.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [57]:
# x_train = x_train.drop(['model year'], axis=1)
# x_train

In [58]:
# test_data = test_data.drop(['model year'], axis=1)


In [59]:
target = ['fuel (L/100km)']
y_train = df[target]    # target variable
y_train

,fuel (L/100km)
0,6.189737
1,15.680667
2,6.817681
3,7.587419
4,14.255152
...,...
293,16.800714
294,5.419585
295,7.259568
296,16.800714


In [60]:
#x_train, x_validation , y_train, y_validation =  train_test_split(x, y, test_size=0.3, random_state=42)

# Now we are done with downloading data! 
* Try building a model inside this notebook by create additional cells below with code to specify and fit the model
* If you are fitting large neural nets, make sure this google colab notebook is running on GPUs
* Check Edit --> Notebook settings --> Hardware accelerator: GPU

In [61]:
# # Start building a model here

# # Step 1: Create a base neural network to which we will add layers
# neural_network = Sequential()

# # Step 2: Add layers to the neural network
# hidden_nodes = 20 # the number of hidden nodes in each layer
# neural_network.add(Dense(hidden_nodes, input_dim=x_train.shape[-1], activation='tanh')) # Input layer
# neural_network.add(Dense(hidden_nodes, activation='tanh'))                              # Hidden layer
# neural_network.add(Dense(y_train.shape[-1], activation='linear'))                           # Output layer

# # Step 3: Create an optimizer, which will be used to train our neural network
# learning_rate = 0.005 # controls the step-size the optimizer takes at each iteration of the training
# optimizer = SGD(lr=learning_rate)
# neural_network.compile(loss='mse', optimizer=optimizer)

# # Step 4: Trainn the neural network
# batch_size = 400  # number of randomly-selected training points for which the optimizer will step
# epochs = 500 # number of training iterations
# history = neural_network.fit(x_train, y_train, batch_size=batch_size, shuffle=True, epochs=epochs, verbose=0)

In [62]:
# # Step 7: Predict label on training set
# y_train_pred = neural_network.predict(x_train)

# # Step 8: Compute MSE on training set
# print('RMSE on Training Data:', sqrt(mean_squared_error(y_train, y_train_pred)))

# # Step 9: Predict label on test set
# #y_test_pred = neural_network.predict(test_data)

# # Step 10: Compute MSE on test set 
# #print('MSE on Testing Data: ', mean_squared_error(y_test, y_test_pred))

In [63]:
# # Step 1: Create a base neural network and regularizes
# neural_network = Sequential()
# reg = l2(0.00000000001) # change the strength of the regularizer

# # Step 2: Add layers to the neural network
# num_hidden_nodes = 10  # the number of hidden nodes in each layer
# neural_network.add(Dense(num_hidden_nodes, input_dim=x_train.shape[-1], activation='tanh', kernel_regularizer=reg, bias_regularizer=reg))
# neural_network.add(Dense(num_hidden_nodes, activation='tanh', kernel_regularizer=reg, bias_regularizer=reg))
# neural_network.add(Dense(y_train.shape[-1], activation='linear', kernel_regularizer=reg, bias_regularizer=reg)) 

# # Step 3: Create an optimizer, which will be used to train our neural network
# learning_rate = 0.001  # controls the step-size the optimizer takes at each iteration of the training
# optimizer = SGD(lr=learning_rate)
# neural_network.compile(loss='mse', optimizer=optimizer)

# # Step 4: Train the neural network
# batch_sz = 500    # number of randomly-selected training points for which the optimizer will step
# num_epochs = 10000  # number of training iterations
# history = neural_network.fit(x_train, y_train, batch_size=batch_sz, shuffle=True, epochs=num_epochs, verbose=0)

In [ ]:
from sklearn import linear_model
import ast

In [64]:
import ast
def process(results):
    """Process results into a dataframe with one column per hyperparameter"""
    
    results = results.copy()
    results['hyperparameters'] = results['hyperparameters'].map(ast.literal_eval)
    
    # Sort with best values on top
    results = results.sort_values('score', ascending = False).reset_index(drop = True)
    
     # Create dataframe of hyperparameters
    hyp_df = pd.DataFrame(columns = list(results.loc[0, 'hyperparameters'].keys()))

    # Iterate through each set of hyperparameters that were evaluated
    for i, hyp in enumerate(results['hyperparameters']):
        hyp_df = hyp_df.append(pd.DataFrame(hyp, index = [0]), 
                               ignore_index = True, sort= True)
        
    # Put the iteration and score in the hyperparameter dataframe
    hyp_df['iteration'] = results['iteration']
    hyp_df['score'] = results['score']
    
    return hyp_df

In [65]:
import random

In [66]:
import lightgbm as lgb

In [67]:
from lightgbm import LGBMRegressor

In [68]:

bmodel = lgb.LGBMRegressor()
bmodel.fit(x_train, y_train)

LGBMRegressor()

In [69]:
from math import sqrt
y_train_pred = bmodel.predict(x_train)
print('MSE on Training Data:', sqrt(mean_squared_error(y_train, y_train_pred)))

MSE on Training Data: 0.8943704087330584


In [70]:
# import xgboost

In [71]:
# best_xgb_model = xgboost.XGBRegressor()
# best_xgb_model.fit(x_train, y_train)

In [72]:
# from sklearn.linear_model import BayesianRidge, LinearRegression
# reg = linear_model.BayesianRidge(compute_score=True)
# reg.fit(x_train, y_train)
# BayesianRidge()

In [73]:
# from math import sqrt

In [74]:
# y_train_pred = best_xgb_model.predict(x_train)
# print('MSE on Training Data:', sqrt(mean_squared_error(y_train, y_train_pred)))



In [75]:
# # from sklearn.ensemble import RandomForestClassifier as RandomForest
# from math import sqrt

In [76]:
y_train=y_train.astype('int')

In [77]:
# # Step 1: instantiate a DecisionTreeClassifier model with maximum depth of your choice
# # Hint: you may want to look at this for reference
# # http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
# # COMPLETE
# rdf = RandomForest(max_depth=50, random_state=0)
# # Step 2: fit the random forest to our data
# # COMPLETE
# rdf.fit(x_train,y_train)
# # C. Predict the labels of the training data
# y_pred1= rdf.predict(x_train)
# print('RMSE on Training Data:', sqrt(mean_squared_error(y_train, y_pred1)))

In [78]:
# # Step 1: Define the polynomial features
# degree = 10
# features = PolynomialFeatures(degree)
# x_train_poly = features.fit_transform(x_train)
# test_data_poly = features.fit_transform(test_data)

# # Step 2: Define the regression model
# linear_regression = LinearRegression(fit_intercept=False)

# # Step 3: Transform the training x's using the polynomial features, and then fit the regression model
# linear_regression.fit(x_train_poly, y_train)

# # Step 5: Predict on training data
# y_train_pred = linear_regression.predict(x_train_poly)
# # Stet 6: Compute train MSE
# print('RMSE on Training Data:', sqrt(mean_squared_error(y_train, y_train_pred)))

# # Step 7: Predict on test data
# #y_test_pred = linear_regression.predict(x_test_poly)
# # Stet 8: Compute test MSE
# #print('MSE on Testing Data: ', mean_squared_error(y_test, y_test_pred))

In [79]:
# # Step 1: Define the polynomial features
# degree = 10
# features = PolynomialFeatures(degree)

# # Step 2: Transform training and test input with PolynomialFeatures
# x_train_poly = features.fit_transform(x_train)
# x_test_poly = features.fit_transform(test_data)

# # Step 3: Instantiate a Ridge regression model
# # Hint: you may want to look at this for reference
# # https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html
# # COMPLETE
# ridge_regression = Ridge(alpha=100000000000000000000000000000)

# # Step 4: Fit the regression model
# # COMPLETE
# ridge_regression.fit(x_train_poly, y_train)

# # Step 5: Compute the train MSE
# # COMPLETE
# y_train_rmse = sqrt(mean_squared_error(y_train, ridge_regression.predict(x_train_poly)))
# print('RMSE on Training Data:', y_train_rmse)

# # Step 6: Compute the test MSE
# # COMPLETE
# # y_test_mse = mean_squared_error(y_test, ridge_regression.predict(x_test_poly))
# # print('MSE on Testing Data:', y_test_mse)

In [80]:
# from sklearn.ensemble import RandomForestRegressor
# regr = RandomForestRegressor()
# regr.fit(x_train, y_train)

In [81]:
# from sklearn.neighbors import KNeighborsClassifier
# classifier = KNeighborsClassifier(n_neighbors=10)
# classifier.fit(x_train, y_train)
# y_train_pred = classifier.predict(x_train)
# print('RMSE on Training Data:', sqrt(mean_squared_error(y_train, y_train_pred)))

In [82]:
# # Step 1: 
# # A. Instantiate an sklearn DecisionTreeRegressor model with maximum depth = 2
# regr1_model = DecisionTreeRegressor(max_depth=200)
# #    Hint: read the documentation https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html
# # B. Fit the model to the training data
# regr1_model.fit(x_train, y_train)
# # C. Predict the labels of the training data
# y_pred_1 = regr1_model.predict(x_train)
# print('RMSE on Training Data:', sqrt(mean_squared_error(y_train, y_pred_1)))

In [83]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   displacement   100 non-null    float64
 1   horsepower     100 non-null    float64
 2   weight         100 non-null    float64
 3   acceleration   100 non-null    float64
 4   model year_70  100 non-null    uint8  
 5   model year_71  100 non-null    uint8  
 6   model year_72  100 non-null    uint8  
 7   model year_73  100 non-null    uint8  
 8   model year_74  100 non-null    uint8  
 9   model year_75  100 non-null    uint8  
 10  model year_76  100 non-null    uint8  
 11  model year_77  100 non-null    uint8  
 12  model year_78  100 non-null    uint8  
 13  model year_79  100 non-null    uint8  
 14  model year_80  100 non-null    uint8  
 15  model year_81  100 non-null    uint8  
 16  model year_82  100 non-null    uint8  
 17  cylinders_4    100 non-null    uint8  
 18  cylinders_8

In [84]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 298 entries, 0 to 297
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   displacement    298 non-null    float64
 1   horsepower      298 non-null    float64
 2   weight          298 non-null    float64
 3   acceleration    298 non-null    float64
 4   fuel (L/100km)  298 non-null    float64
 5   model year_70   298 non-null    uint8  
 6   model year_71   298 non-null    uint8  
 7   model year_72   298 non-null    uint8  
 8   model year_73   298 non-null    uint8  
 9   model year_74   298 non-null    uint8  
 10  model year_75   298 non-null    uint8  
 11  model year_76   298 non-null    uint8  
 12  model year_77   298 non-null    uint8  
 13  model year_78   298 non-null    uint8  
 14  model year_79   298 non-null    uint8  
 15  model year_80   298 non-null    uint8  
 16  model year_81   298 non-null    uint8  
 17  model year_82   298 non-null    uin

In [85]:
test_data = pd.DataFrame(test_data)

# Once you have generated predictions on the holdout and test data, create a submission file
Your submission file should follow the Kaggle submission template.


In [ ]:
# Here's a sample submission file
sample_submission = pd.read_csv('https://raw.githubusercontent.com/onefishy/Rwanda-course-2020/master/Competition_data/sampleSubmission.csv') # read in the data as a DataFrame
sample_submission.head(3) # show the first 3 rows of the dataset

,id,predictions
0,0,22.661621
1,1,6.381661
2,2,21.673614


In [ ]:
#index_col = np.arange(100)
#index_col = list(index_col)

In [ ]:
predictions = bmodel.predict(test_data)
predictions

array([ 9.10320383, 11.67992622,  5.96223793, 18.11442207, 18.11442207,
        9.27934146,  7.38630675, 10.39175775,  9.09563339,  9.29678069,
       10.91105938,  8.68486588,  7.24313525,  7.01854776, 13.98979235,
       15.5081781 , 12.84945045, 11.85947638, 10.33993998, 10.84227226,
       17.19092936, 11.68381075,  7.11028767, 13.61487789, 17.27082545,
        7.55842158,  9.18114339,  8.77061693, 14.88093564, 19.61337726,
       13.187559  ,  6.32801323, 10.32103909,  8.97053447,  9.96966723,
       16.19029776,  7.96426352,  8.24892102, 17.21744009, 10.65709506,
        6.96157077, 17.21744009,  9.55918216, 16.24775401, 12.42382095,
       13.57498972,  8.39039262, 10.44723015, 12.4860897 ,  6.62649578,
       12.53722256, 12.11789139, 10.76206118,  9.32305659, 14.26404224,
        9.47900092,  6.61937422,  8.15778275, 13.68288847,  6.93158797,
        8.84020318,  7.35137783,  7.41950149, 13.76688344, 13.01925471,
       11.84147585,  7.76926251,  7.13744283, 18.44952694, 15.91

In [ ]:

# Save submission file
id=np.arange(0,100)
submission_sample = pd.DataFrame(predictions,id)
submission_sample.to_csv('submission_sample.csv')
dff=pd.read_csv('submission_sample.csv')
dff.columns=dff.columns.str.strip().str.replace(' ', '_')
submision34=dff.rename(columns={'Unnamed:_0':'id','0':'predictions'})
submision34.to_csv('submision34.csv',index=False) 

# Save and download the submission file and upload to the Kaggle website

Then download `my_submission.csv` by running the following line and submit to the [Kaggle](https://www.kaggle.com/t/b9bc778c9e8842d28c5526f578e6c348) compeition website.

In [ ]:
files.download('submision34.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>